<a href="https://colab.research.google.com/github/vichacker1234/Proyectos_IA/blob/main/sistema_de_recomendacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## *Sistema de recomendación usando word2vec*



In [46]:
#Se importan las librerás más importantes
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from gensim.models import Word2Vec
import random
import warnings;
warnings.filterwarnings('ignore')

In [23]:
#Lectura del dataframe
datos=pd.read_excel('/content/Online_Retail.xlsx')


In [24]:
datos.head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 08:26:00,7.65,17850.0,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01 08:26:00,4.25,17850.0,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047.0,United Kingdom


## *Descripción de las variables*
* InvoiceNo: Número de factura. un número único asignado a cada transacción
* StockCode: Código de producto/artículo. un número único asignado a cada producto distinto
* Description: Descripción del producto
* Quantity: Las cantidades de cada producto por transacción
* InvoiceDate: Fecha y hora de la factura. El día y la hora en que se generó cada transacción
* CustomerID: Número de cliente. Un número único asignado a cada cliente


In [26]:
#Número de observaciones y de variables 
datos.shape

(541909, 8)

El modelo tiene 541909 transacciones o registros ques es bastante bueno para entrenar el modelo

In [27]:
#Ahora se va a analizar si hay datos perdidos (NAs)
datos.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

Aquí se puede apreciar que hay datos nulos

In [28]:
datos.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

Como el tamaño de la muestra de registros es bastante grande, se procede a eliminar estas observaciones

In [29]:
#Remover o eliminar nas
datos.dropna(inplace=True)

In [30]:
#Se revisa nuevamente que no haya nas
datos.isna().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

## Preparación del conjunto de datos
 Se va a cambiar el tipo de stockcode a string

In [34]:
#Se cambia 
datos['StockCode']=datos['StockCode'].astype('str')

In [35]:
type(datos['StockCode'])

pandas.core.series.Series

Se va a revisar el número de clientes únicos en este conjunto de datos 

In [36]:
customers = datos["CustomerID"].unique().tolist()
len(customers)

4372

En nuestro conjunto de datos hay 4.372 clientes. Para cada uno de estos clientes, extraeremos su historial de compras. En otras palabras, podemos tener 4.372 secuencias de compras.

A partir de estas condiciones se va a establecer un conjunto de entranamiento y uno de test

In [38]:
#Se va a tomar el 80% del conjunto de datos para entrenamiento y el 20 % para validación

#Elegir las identificaciones de los clientes
random.shuffle(customers)

# extraer el  80% de los ID's de los clientes 
customers_train = [customers[i] for i in range(round(0.8*len(customers)))]

# dividir los datos en conjunto de entrenamiento y validación
datos_entrenamiento = datos[datos['CustomerID'].isin(customers_train)]
datos_validacion = datos[~datos['CustomerID'].isin(customers_train)]



Se va a crear una secuencia de compras realizadas por los clientes en el conjunto de datos tanto para el conjunto de entrenamiento como para el de validación.

In [39]:
# lista para capturar el historial de compras de los clientes
compra_entrenamiento = []

# rellenar la lista con los códigos de los productos
for i in tqdm(customers_train):
    temp = datos_entrenamiento[datos_entrenamiento["CustomerID"] == i]["StockCode"].tolist()
    compra_entrenamiento.append(temp)

100%|██████████| 3498/3498 [00:03<00:00, 891.89it/s]


In [41]:
# lista para capturar el historial de compras de los clientes
compra_validacion = []

# populate the list with the product codes
for i in tqdm(datos_validacion['CustomerID'].unique()):
    temp = datos_validacion[datos_validacion["CustomerID"] == i]["StockCode"].tolist()
    compra_validacion.append(temp)

100%|██████████| 874/874 [00:00<00:00, 1253.33it/s]


## Construir incrustaciones (embeddings) word2vec para los productos

In [51]:
# Entrenar el modelo word2vec
modelo_word2vec = Word2Vec(window = 10, sg = 1, hs = 0,
                 negative = 10, # for negative sampling
                 alpha=0.03, min_alpha=0.0007,
                 seed = 14)

modelo_word2vec.build_vocab(compra_entrenamiento, progress_per=200)

modelo_word2vec.train(compra_entrenamiento, total_examples = model.corpus_count, 
            epochs=10, report_delay=1)

(3210152, 3245260)

Como no tenemos previsto entrenar más el modelo, llamamos aquí a init_sims( ). Esto hará que el modelo sea mucho más eficiente en cuanto a memoria:

In [53]:
modelo_word2vec.init_sims(replace=True)

Veamos un resumen del modelo que se ha creado

In [54]:
print(modelo_word2vec)

Word2Vec(vocab=3132, size=100, alpha=0.03)


Nuestro modelo tiene un vocabulario de 3.151 palabras únicas y sus vectores de tamaño 100 cada uno. A continuación, extraeremos los vectores de todas las palabras de nuestro vocabulario y los almacenaremos en un solo lugar para facilitar su acceso.

In [57]:
# Se extraen todos los vectores ( se guarda como vecs)
vecs = modelo_word2vec[modelo_word2vec.wv.vocab]

vecs.shape

(3132, 100)

Siempre es muy útil visualizar las incrustaciones que has creado. Aquí tenemos incrustaciones de 100 dimensiones. Ni siquiera podemos visualizar 4 dimensiones y mucho menos 100. ¿Qué podemos hacer?

Vamos a reducir las dimensiones de las incrustaciones del producto de 100 a 2 utilizando el algoritmo UMAP. Este algoritmo es popularmente utilizado para la reducción de la dimensionalidad.

 Ahora, nuestro siguiente paso es sugerir productos similares para un determinado producto o un vector de productos.

In [66]:
productos = datos_entrenamiento[["StockCode", "Description"]]

# remover duplicados
productos.drop_duplicates(inplace=True, subset='StockCode', keep="last")

# crear un diccionario de ID de producto y descripción de producto
productos_dict = productos.groupby('StockCode')['Description'].apply(list).to_dict()

In [70]:
#Revisar el diccionario
# test the dictionary
productos_dict['71053']

['WHITE MOROCCAN METAL LANTERN']

Se ha definido la función siguiente. Tomará un vector de productos (n) como entrada y devolverá los 6 mejores productos similares:*texto en cursiva*

In [71]:
#Se define la función similar_productos
def similar_productos(v, n = 6):
    
    # extract most similar products for the input vector
    ms = modelo_word2vec.similar_by_vector(v, topn= n+1)[1:]
    
    # extract name and similarity score of the similar products
    new_ms = []
    for j in ms:
        pair = (productos_dict[j[0]][0], j[1])
        new_ms.append(pair)
        
    return new_ms        

Probemos nuestra función pasando el vector del producto '90019A'(‘SILVER M.O.P ORBIT BRACELET’):

In [73]:
similar_productos(modelo_word2vec['90019A'])

[('AMBER DROP EARRINGS W LONG BEADS', 0.7762057185173035),
 ('JADE DROP EARRINGS W FILIGREE', 0.7747208476066589),
 ('SILVER M.O.P. ORBIT NECKLACE', 0.7658032774925232),
 ('SILVER HOOP EARRINGS WITH FLOWER', 0.7551912665367126),
 ('SILVER ROCCOCO CHANDELIER', 0.7453089356422424),
 ('AMBER GLASS/SILVER BRACELET', 0.7451914548873901)]

Los resultados son bastante relevantes y se ajustan bien al producto de entrada. Sin embargo, este resultado se basa en el vector de un solo producto. ¿Qué pasa si queremos recomendar productos basados en las múltiples compras que el usuario ha hecho en el pasado?


Una solución sencilla es tomar la media de todos los vectores de los productos que el usuario ha comprado hasta ahora y utilizar este vector resultante para encontrar productos similares. Utilizaremos la siguiente función que toma una lista de ID de productos y da un vector de 100 dimensiones que es una media de los vectores de los productos de la lista de entrada:

In [76]:
#Se define la función vectores_agregados
def vectores_agregados(productos):
    product_vec = []
    for i in productos:
        try:
            product_vec.append(modelo_word2vec[i])
        except KeyError:
            continue
        
    return np.mean(product_vec, axis=0)

Recordemos que ya hemos creado una lista separada de secuencias de compra para fines de validación. Ahora vamos a utilizarla.


In [78]:
len(compra_validacion[0])

1011

La longitud de la primera lista de productos comprados por un usuario es 1011. Pasaremos esta secuencia de productos del conjunto de validación a la función 'vectores_agregados'.

In [79]:
vectores_agregados(compra_validacion[0]).shape

(100,)

Bien, la función ha devuelto un array de 100 dimensiones. Esto significa que la función está funcionando bien. Ahora podemos utilizar este resultado para obtener los productos más similares:

In [80]:
similar_productos(vectores_agregados(compra_validacion[0]))

[('RECIPE BOX WITH METAL HEART', 0.6766895055770874),
 ('POLYESTER FILLER PAD 65CMx65CM', 0.660896897315979),
 ('REGENCY CAKESTAND 3 TIER', 0.6564865708351135),
 ('SET/5 RED RETROSPOT LID GLASS BOWLS', 0.6525031328201294),
 ('WOOD BLACK BOARD ANT WHITE FINISH', 0.6455643177032471),
 ('WHITE HANGING HEART T-LIGHT HOLDER', 0.6395620107650757)]

Resulta que nuestro sistema ha recomendado 6 productos basándose en todo el historial de compras de un usuario. Además, si desea obtener sugerencias de productos basadas en las últimas compras, sólo entonces también puede utilizar el mismo conjunto de funciones.

A continuación, estoy dando sólo los últimos 10 productos comprados como entrada:

In [82]:
similar_productos(vectores_agregados(compra_validacion[1][-10:]))

[('ALARM CLOCK BAKELIKE GREEN', 0.7768344879150391),
 ('ALARM CLOCK BAKELIKE RED ', 0.7598289847373962),
 ('ALARM CLOCK BAKELIKE CHOCOLATE', 0.7572663426399231),
 ('ALARM CLOCK BAKELIKE PINK', 0.7219333052635193),
 ('ALARM CLOCK BAKELIKE ORANGE', 0.664379358291626),
 ('BAG 250g SWIRLY MARBLES', 0.5926131010055542)]